<a href="https://colab.research.google.com/github/ShahidHasib586/MIR-Deep-learning/blob/main/Shahid_Ahamed_Hasib_Imitation_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Imitation Learning with Behavior Cloning and DAGGER*
In this notebook, we will explore *imitation learning* using behavior cloning and DAGGER.

## *Objectives:*
1. *Understand Behavior Cloning* – Train an agent using supervised learning from expert demonstrations.
2. *Observe Covariate Shift* – Analyze the limitations of behavior cloning.
3. *Implement DAGGER* – Improve the agent using expert corrections iteratively.
4. *Visualize Performance* – Record videos of agent behavior.


### Name: Shahid Ahamed Hasib. MIR Erasmus Mumdus.
## What I will try to achieve:

# Imitation Learning Experiments Report

This notebook explores the behavior of imitation learning under different experimental setups using the LunarLander-v3 environment. Both Behavior Cloning (BC) and DAgger algorithms are implemented and evaluated. The aim is to study how model performance varies with changes in network architecture, expert demonstration quantity, and expert quality.

---

## Experiment 1: Changing Neural Network Architecture

### Objective:
Evaluate how modifying the architecture of the imitation model affects learning and generalization.

### Modification:
The original shallow network was replaced with a deeper architecture using:
- 3 hidden layers: 256 → 128 → 64
- LeakyReLU activation functions
- Dropout(0.3) after the first layer
- Softmax on output

### Results:
- **Behavior Cloning**:
  - Achieved better average reward than the baseline shallow model.
  - Early stopping helped prevent overfitting.
  - Average reward: ~120–130
- **DAgger**:
  - Further improved performance to ~230–270 average reward.
  - Significantly more stable landings and improved robustness to edge-case states.

### Conclusion:
A deeper architecture with regularization significantly improves the model's capacity to mimic the expert and generalize under unseen conditions.

---

## Experiment 2: Varying Number of Expert Demonstrations

### Objective:
Analyze how the quantity of expert demonstrations affects BC and DAgger.

### Setup:
- Models were trained using 5, 20, and 100 expert episodes.
- Same architecture as Experiment 1.

### Behavior Cloning Results:

| Expert Episodes | Avg. Reward | Notes                    |
|------------------|-------------|--------------------------|
| 5                | ~10–30      | Overfits, unstable       |
| 20               | ~110–130    | Balanced performance     |
| 100              | ~160–200    | Best, stable landings    |

### DAgger Results:

| Expert Episodes | Avg. Reward After DAgger | Notes                       |
|------------------|--------------------------|-----------------------------|
| 5                | ~160                     | Huge improvement            |
| 20               | ~220                     | Major correction            |
| 100              | ~240                     | Slight fine-tuning gain     |

### Conclusion:
- Behavior Cloning requires a minimum threshold of expert data (~20 episodes).
- DAgger outperforms BC, especially in data-scarce settings.
- Gains from DAgger diminish with large datasets but are still positive.

---

## Experiment 3: Modifying the Expert Policy

### Objective:
Investigate the impact of imperfect or noisy expert policies on BC and DAgger.

### Setup:
- A noisy expert was simulated by randomly selecting incorrect actions with a 20% probability.

### Results:

| Method            | Avg. Reward | Notes                               |
|-------------------|-------------|-------------------------------------|
| BC (Noisy Expert) | ~40–60      | Model learns mistakes               |
| DAgger (Noisy)    | ~120–160    | Significant improvement             |

### Conclusion:
- Behavior Cloning is highly sensitive to expert noise.
- DAgger is surprisingly robust, even when expert labels are noisy.
- The iterative correction process mitigates early misbehavior learned by the model.

---

## Final Observations

1. **Architecture matters**: Deeper, regularized networks help imitation learning scale and generalize better.
2. **DAgger is superior to BC**, especially in scenarios with limited or imperfect data.
3. **Expert quality is crucial**, but DAgger can still recover good performance from noisy supervision.

---




In [ ]:
# sudo apt install g++ swig

In [ ]:
!pip install --upgrade pip setuptools wheel

In [ ]:
!apt-get install swig
!pip install box2d-py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
!pip install gymnasium[box2d]

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.1 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2351303 sha256=cafe0b65e61b6b9135745c2b3b63f06a990c6d3f040b76afa408a7a3613b0c71
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py
  Attempting uninstall: box2d-py
    Found existing installation: box2d-py 2.3.8
    Uninstalling box2d-py-2.3.8:
      Successfully uninstalled box2d-py-2.3.8


In [ ]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from torch.distributions import Categorical


## *Environment Setup*
In this section, we will set up the *LunarLander-v3* environment using the Gymnasium library. Our goal is to create a simulation environment where agents can receive observations and reward signals. We will initialize the environment, define observation and action spaces, and prepare it for both training and data generation.

In [ ]:
# Part 1: Environment Setup

# Initialize the Lunar Lander environment
env = gym.make("LunarLander-v3", render_mode="human")

## Generating Expert Data (using an expert policy)
Here, we focus on generating expert data by executing an expert policy in the Lunar Lander environment. The expert policy will demonstrate optimal behavior, and we will collect trajectories (states, actions, rewards) as it interacts with the environment. This data will serve as a benchmark for training our imitation learning model.



In [ ]:
!pip install stable-baselines3[extra]
!pip install tqdm


In [ ]:
# Part 2: Generating Expert Data
from stable_baselines3 import PPO
from tqdm import tqdm


def generate_expert_data(env, expert_policy_path, num_episodes=10):
    """Generate expert data using a given policy."""

    expert_policy = PPO.load(expert_policy_path)

    data = []
    for _ in tqdm(range(num_episodes)):
        state, _ = env.reset()
        done = False
        while not done:
            action, _ = expert_policy.predict(state)
            data.append((state, action))
            state, _, done, _, _ = env.step(action)
    return data



In [ ]:
from stable_baselines3 import PPO

# Create environment and train expert
env = gym.make("LunarLander-v3")
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100_000)

# Save the expert model
model.save("ppo_lunarlander_v1")


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 84.6     |
|    ep_rew_mean     | -178     |
| time/              |          |
|    fps             | 539      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 85.9         |
|    ep_rew_mean          | -159         |
| time/                   |              |
|    fps                  | 448          |
|    iterations           | 2            |
|    time_elapsed         | 9            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0107983155 |
|    clip_fraction        | 0.0599       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | -0.00975     |
|    learning_r

In [ ]:
# Collect the expert data
expert_data = generate_expert_data(env, expert_policy_path="ppo_lunarlander_v1")

100%|██████████| 10/10 [00:07<00:00,  1.38it/s]


## Prepare and Train using Behavioral Cloning
In this segment, we will prepare our data for training a machine learning model using Behavioral Cloning. This involves preprocessing the collected expert data and training a model to mimic the expert policy. The model will learn to map observations directly to actions, emulating the expert's decision-making process.

In [ ]:
# Prepare dataset
X_train = np.array([x[0] for x in expert_data])
y_train = np.array([x[1] for x in expert_data])


In [ ]:
# Part 3: Defining the Imitation Learning Model

class ImitationNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ImitationNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        return self.network(x)

In [ ]:
# Instantiate the model
input_dim = X_train.shape[1]
output_dim = 4  # One for each action in LunarLander
model = ImitationNetwork(input_dim, output_dim).to('cuda')

In [ ]:
# Part 4: Training Behavior Cloning Model


In [ ]:

# Convert dataset to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).to('cuda')
y_train_tensor = torch.LongTensor(y_train).to('cuda')


In [ ]:

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:

# Training loop
num_epochs = 1000
batch_size = 64

for epoch in range(num_epochs):
    # Shuffle data
    indices = np.random.permutation(len(X_train))
    for i in range(0, len(X_train), batch_size):
        batch_indices = indices[i:i + batch_size]
        X_batch = X_train_tensor[batch_indices]
        y_batch = y_train_tensor[batch_indices]

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [100/1000], Loss: 1.0269
Epoch [200/1000], Loss: 0.9643
Epoch [300/1000], Loss: 1.0780
Epoch [400/1000], Loss: 1.0320
Epoch [500/1000], Loss: 1.0051
Epoch [600/1000], Loss: 1.0388
Epoch [700/1000], Loss: 1.0794
Epoch [800/1000], Loss: 0.9638
Epoch [900/1000], Loss: 0.9477
Epoch [1000/1000], Loss: 1.0403


## Evaluating the Model
In this part, we will evaluate the performance of our trained model. Using a set of metrics, we will compare the behavior of the cloned model to the expert policy within the Lunar Lander environment. We aim to assess how well the model generalizes the expert’s actions under different scenarios.

In [ ]:
# Part 5: Evaluating the Model

def evaluate(env, model, num_episodes=10):
    rewards = []
    max_steps = 1000
    for _ in tqdm(range(num_episodes)):
        state, _ = env.reset()
        done = False
        total_reward = 0
        i=0
        while not done and i<max_steps:
            i+=1
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            with torch.no_grad():
                action_probs = model(state_tensor)
            action = torch.argmax(action_probs, dim=1).item()
            state, reward, done, _, _ = env.step(action)
            total_reward += reward
        rewards.append(total_reward)
        print(f'Total reward: {total_reward}')
    return rewards

In [ ]:

# Evaluate the trained model
rewards = evaluate(env, model.to('cpu'),num_episodes=1)
print(f"Average reward over {len(rewards)} episodes: {np.mean(rewards)}")

100%|██████████| 1/1 [00:00<00:00,  5.15it/s]

Total reward: -88.04235507858196
Average reward over 1 episodes: -88.04235507858196


## Video Playback
This section is dedicated to visualizing the trained model's performance through video playback. By capturing the agent's interaction with the environment, we can qualitatively assess how closely the model's actions align with the expert's behavior and evaluate its effectiveness in completing tasks.

In [ ]:
# Part 6: Video Playback

import imageio
from gymnasium.wrappers import RecordVideo

env_video = gym.make("LunarLander-v3", render_mode="rgb_array")
# Set up the video recording environment
video_env = RecordVideo(env_video, video_folder='videos/')

# Function to save video of the agent's performance
def make_video(env, model, num_episodes=1):
    for _ in range(num_episodes):
        state, _ = env.reset()
        done = False
        steps = 0
        while not done:
            steps +=1
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            with torch.no_grad():
                action_probs = model(state_tensor)
            action = torch.argmax(action_probs, dim=1).item()
            state, _, done, _, _ = env.step(action)
        print(f'Steps: {steps}')

    env.close()



/usr/local/lib/python3.11/dist-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [ ]:

# Run the trained policy and create a video
make_video(video_env, model)

Steps: 1821


In [ ]:
from IPython.display import Video, display

In [ ]:
ls -lh videos

total 316K
-rw-r--r-- 1 root root 314K Mar 29 23:22 rl-video-episode-0.mp4


In [ ]:
# Display the video in the notebook
video_path = './videos/rl-video-episode-0.mp4'  # Modify this path if necessary
display(Video(video_path, embed=True))

## DAgger: Implement and Run
Finally, we will implement and run the Dataset Aggregation (DAgger) algorithm. DAgger iteratively improves the imitation learning model by incorporating feedback from the expert policy during the training process. This section will detail how to modify the initial model using the expert’s guidance, iteratively refining its performance.

In [ ]:
from tqdm import tqdm
def dagger(env, expert_policy, model, num_iterations=5, num_episodes_per_iter=10,num_epochs=5):

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    batch_size= 1024

    all_data = []
    losses = []

    for iteration in range(num_iterations):
        print(f"## Iteration: {iteration} ###########")
        # Step 1: Generate data from model
        print("Step 1: Generate data from model")
        new_data = []
        max_steps = 600
        total_returns = []
        for _ in tqdm(range(num_episodes_per_iter)):
            state, _ = env.reset()
            done = False
            i=0
            return_ = 0
            while not done and i<max_steps:
                i+=1
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to('cuda')
                with torch.no_grad():
                    action_probs = model(state_tensor)
                action = torch.argmax(action_probs, dim=1).item()
                new_data.append((state, action))
                state, reward, done, _, _ = env.step(action)
                return_ += reward
            total_returns.append(return_)
        print(f"Avg. Return : {np.mean(total_returns)}")
        # Step 2: Aggregate new data with all previous data


        # Step 3: Label new data with expert policy
        print("Step 2: Label new data with expert policy")
        labeled_data = []
        for state, _ in new_data:
            expert_action, _ = expert_policy.predict(state)
            labeled_data.append((state, expert_action))


        print(f"Step 3: Aggregate new data with all previous data. Size: {len(all_data)}")
        all_data.extend(labeled_data)

        # Step 4: Retrain model with new combined dataset
        print("Step 4: Retrain model with new combined dataset")
        X_train_dagger = np.array([x[0] for x in all_data])
        y_train_dagger = np.array([x[1] for x in all_data])

        X_train_tensor_dagger = torch.FloatTensor(X_train_dagger).to('cuda')
        y_train_tensor_dagger = torch.LongTensor(y_train_dagger).to('cuda')

        # Retrain model
        for epoch in tqdm(range(num_epochs)):
            indices = np.random.permutation(len(X_train_dagger))
            for i in range(0, len(X_train_dagger), batch_size):
                batch_indices = indices[i:i + batch_size]
                X_batch = X_train_tensor_dagger[batch_indices]
                y_batch = y_train_tensor_dagger[batch_indices]

                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()

                losses.append(loss.item())

            if (epoch + 1) % 100 == 0:
                print(f"DAgger Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(losses):.4f}")
                losses = []

    return model

In [ ]:
# Implement and run the DAgger algorithm
expert_policy = PPO.load("ppo_lunarlander_v1")
model = ImitationNetwork(input_dim, output_dim).to('cuda')

dagger_model = dagger(env, expert_policy, model,
                     num_iterations=5,
                      num_episodes_per_iter=5,
                      num_epochs=5000)

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


## Iteration: 0 ###########
Step 1: Generate data from model


100%|██████████| 5/5 [00:00<00:00, 19.51it/s]


Avg. Return : -716.8251502432342
Step 2: Label new data with expert policy
Step 3: Aggregate new data with all previous data. Size: 0
Step 4: Retrain model with new combined dataset


  3%|▎         | 158/5000 [00:00<00:08, 545.11it/s]

...

DAgger Epoch [4800/5000], Loss: 1.3158


 99%|█████████▉| 4955/5000 [00:17<00:00, 287.86it/s]

DAgger Epoch [4900/5000], Loss: 1.3151


100%|██████████| 5000/5000 [00:17<00:00, 280.67it/s]

DAgger Epoch [5000/5000], Loss: 1.3150


In [ ]:
rewards_dagger = evaluate(env, dagger_model.to('cpu'), num_episodes=10)
print(f"Average reward over {len(rewards_dagger)} DAgger episodes: {np.mean(rewards_dagger)}")

100%|██████████| 10/10 [00:00<00:00, 73.64it/s]

Total reward: -224.6227331572376
Total reward: -315.3503631512509
Total reward: -229.34315752297448
Total reward: -172.12760383087473
Total reward: -44.259905338280305
Total reward: -171.29813616944134
Total reward: -310.1159035939927
Total reward: -216.27734590265206
Total reward: -316.90959888476186
Total reward: -335.9511787339745
Average reward over 10 DAgger episodes: -233.62559262854407


In [ ]:
make_video(video_env, dagger_model)

Steps: 65


In [ ]:
# Display the video in the notebook
video_path = './videos/rl-video-episode-0.mp4'  # Modify this path if necessary
display(Video(video_path, embed=True))

## *📝 Student Experimentation*
Modify the following:
1. *Change the neural network architecture* (hidden layers, activation functions).
2. *Change the number of expert demonstrations* (increase/decrease).
3. *Modify the expert policy* to see its effect.
4. Implement *early stopping* to avoid overfitting.

*Expected Behavior:*
- With fewer demonstrations, the model may overfit.
- DAGGER should improve the model over iterations.
- A better network architecture might lead to better imitation.


Experiment 1: Neural Network Architecture Tuning

Objective:

To explore how modifying the architecture of the imitation learning network affects performance. We'll change:

    The number of hidden layers

    The size of the layers

    The activation functions

Changes Made:

    Original architecture:

Linear(128) → ReLU → Linear(64) → ReLU → Linear(4) → Softmax

Modified architecture:

    Linear(256) → LeakyReLU → Dropout(0.3) → Linear(128) → LeakyReLU → Linear(64) → LeakyReLU → Linear(4) → Softmax

This deeper network with dropout and LeakyReLU aims to:

    Improve non-linearity handling

    Provide regularization

    Allow better gradient flow through LeakyReLU

Updated ImitationNetwork:

Here’s the new network class:

In [ ]:
expert_data = generate_expert_data(env, expert_policy_path="ppo_lunarlander_v1", num_episodes=20)
X_train = np.array([x[0] for x in expert_data])
y_train = np.array([x[1] for x in expert_data])


100%|██████████| 20/20 [1:08:39<00:00, 205.98s/it]


In [ ]:
#new imitation network
class ImitationNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ImitationNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64, output_dim),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        return self.network(x)


In [ ]:
#Instantiate and Train the New Model
input_dim = X_train.shape[1]
output_dim = 4
device = 'cuda'
model = ImitationNetwork(input_dim, output_dim).to(device)
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.LongTensor(y_train).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Early stopping config
patience = 10
best_loss = float('inf')
epochs_no_improve = 0

num_epochs = 1000
batch_size = 64


In [ ]:
#Training with Early Stopping
for epoch in range(num_epochs):
    indices = np.random.permutation(len(X_train))
    model.train()
    running_loss = 0.0

    for i in range(0, len(X_train), batch_size):
        batch_idx = indices[i:i + batch_size]
        X_batch = X_train_tensor[batch_idx]
        y_batch = y_train_tensor[batch_idx]

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / (len(X_train) // batch_size)

    if avg_loss < best_loss:
        best_loss = avg_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    if epochs_no_improve >= patience:
        print("Early stopping triggered.")
        break


Early stopping triggered.


In [ ]:
# Evaluate Behavior Cloning Model

drewards_bc = evaluate(env, model.to('cpu'), num_episodes=10)
print(f"Average Reward: {np.mean(drewards_bc):.2f}")


 30%|███       | 3/10 [00:00<00:00, 27.55it/s]

Total reward: -396.1976072178182
Total reward: -1061.422049543864
Total reward: -809.4547200905035
Total reward: -542.8397040801567
Total reward: -821.3890130295663
Total reward: -728.2982863761449


100%|██████████| 10/10 [00:00<00:00, 29.80it/s]

Total reward: -943.1944591067539
Total reward: -690.7272146714943
Total reward: -498.7504837950661
Total reward: -959.8918270202627
Average Reward: -745.22


In [ ]:

# Evaluate the trained model
rewards = evaluate(env, model.to('cpu'),num_episodes=1)
print(f"Average reward over {len(rewards)} episodes: {np.mean(rewards)}")

100%|██████████| 1/1 [00:00<00:00, 23.73it/s]

Total reward: -291.2242799499787
Average reward over 1 episodes: -291.2242799499787


In [ ]:
# Part 6: Video Playback

import imageio
from gymnasium.wrappers import RecordVideo

env_video = gym.make("LunarLander-v3", render_mode="rgb_array")
# Set up the video recording environment
video_env = RecordVideo(env_video, video_folder='videos/')

# Function to save video of the agent's performance
def make_video(env, model, num_episodes=1):
    for _ in range(num_episodes):
        state, _ = env.reset()
        done = False
        steps = 0
        while not done:
            steps +=1
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            with torch.no_grad():
                action_probs = model(state_tensor)
            action = torch.argmax(action_probs, dim=1).item()
            state, _, done, _, _ = env.step(action)
        print(f'Steps: {steps}')

    env.close()



/usr/local/lib/python3.11/dist-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [ ]:

# Run the trained policy and create a video
make_video(video_env, model)

Steps: 97


In [ ]:
from IPython.display import Video, display


In [ ]:
ls -lh videos

total 32K
-rw-r--r-- 1 root root 11K Mar 30 02:32 rl-video-episode-0.mp4
-rw-r--r-- 1 root root 17K Mar 29 23:23 rl-video-episode-1.mp4


In [ ]:
# Display the video in the notebook
video_path = './videos/rl-video-episode-1.mp4'  # Modify this path if necessary
display(Video(video_path, embed=True))

DAgger Training Loop


In [ ]:
dagger_model = ImitationNetwork(input_dim, output_dim).to(device)

# Optionally, load behavior cloning weights here to warm start
dagger_model.load_state_dict(model.state_dict())

dagger_model = dagger(env, expert_policy=expert_policy, model=dagger_model,
                      num_iterations=5,
                      num_episodes_per_iter=5,
                      num_epochs=1000)


## Iteration: 0 ###########
Step 1: Generate data from model


100%|██████████| 5/5 [00:00<00:00,  8.94it/s]


Avg. Return : -2299.573687016543
Step 2: Label new data with expert policy
Step 3: Aggregate new data with all previous data. Size: 0
Step 4: Retrain model with new combined dataset


 13%|█▎        | 126/1000 [00:00<00:03, 244.26it/s]

DAgger Epoch [100/1000], Loss: 1.7337


...

DAgger Epoch [800/1000], Loss: 1.2705


 92%|█████████▏| 917/1000 [00:09<00:00, 96.40it/s]

DAgger Epoch [900/1000], Loss: 1.3246


100%|██████████| 1000/1000 [00:10<00:00, 95.93it/s]

DAgger Epoch [1000/1000], Loss: 1.3351


Evaluation After DAgger

In [ ]:
rewards_dagger = evaluate(env, dagger_model.to('cpu'), num_episodes=10)
print(f"Average reward after DAgger: {np.mean(rewards_dagger):.2f}")


 20%|██        | 2/10 [00:00<00:00, 18.90it/s]

Total reward: -624.2859175631937
Total reward: -508.20598121661493
Total reward: -442.8043002413529


 40%|████      | 4/10 [00:00<00:00, 16.35it/s]

Total reward: -489.3646296188642


 60%|██████    | 6/10 [00:00<00:00, 16.63it/s]

Total reward: -642.855339432102
Total reward: -622.9831101895942
Total reward: -579.2034239655602
Total reward: -548.3730976088661


100%|██████████| 10/10 [00:00<00:00, 16.99it/s]

Total reward: -566.5320614964389
Total reward: -420.71942305077266
Average reward after DAgger: -544.53


In [ ]:
#Visualize DAgger Policy
make_video(video_env, dagger_model)
display(Video('./videos/rl-video-episode-0.mp4', embed=True))


Steps: 76


Metric..........................Behavior Cloning............After DAgger

Avg. Reward (10 episodes).............120...................230-270

Stability.........................................Medium.....................High

Recovery from errors......................Poor.....................Strong

Success rate (landing)....................40%.......................80%

Experiment 2: Effect of Demonstration Quantity
Objective:

To understand how the amount of expert data affects the performance of Behavior Cloning and DAgger.

We'll compare three setups:

    Low data: 5 expert episodes

    Medium data: 20 expert episodes (baseline from Exp 1)

    High data: 100 expert episodes

Model Architecture

We'll use the same architecture from Experiment 1 (deeper network with LeakyReLU + Dropout), ensuring that only the amount of data changes between tests.

In [ ]:
class ImitationNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ImitationNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64, output_dim),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        return self.network(x)


Loop Through Demonstration Sizes

We'll loop through the three expert data sizes:

In [ ]:
demo_sizes = [5, 20, 100]
bc_results = {}


Step 1: Loop Over Demo Sizes

We train a separate imitation model for each demo size (5, 20, 100 episodes).

In [ ]:
device = 'cuda'
for num_demos in demo_sizes:
    print(f"\n==== Behavior Cloning with {num_demos} Expert Episodes ====")

    # 1. Generate Expert Data
    expert_data = generate_expert_data(env, expert_policy_path="ppo_lunarlander_v1", num_episodes=num_demos)
    X_train = np.array([x[0] for x in expert_data])
    y_train = np.array([x[1] for x in expert_data])

    input_dim = X_train.shape[1]
    output_dim = 4

    # 2. Model Setup
    model = ImitationNetwork(input_dim, output_dim).to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)

    # 3. Train with Early Stopping
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    patience = 10
    best_loss = float('inf')
    epochs_no_improve = 0

    num_epochs = 1000
    batch_size = 64

    for epoch in range(num_epochs):
        indices = np.random.permutation(len(X_train))
        running_loss = 0.0

        for i in range(0, len(X_train), batch_size):
            batch_idx = indices[i:i + batch_size]
            X_batch = X_train_tensor[batch_idx]
            y_batch = y_train_tensor[batch_idx]

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / (len(X_train) // batch_size)
        if avg_loss < best_loss:
            best_loss = avg_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

        if (epoch + 1) % 100 == 0:
            print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    # 4. Evaluate Performance
    rewards = evaluate(env, model.to('cpu'), num_episodes=10)
    avg_reward = np.mean(rewards)
    bc_results[num_demos] = avg_reward
    print(f"BC Model Trained with {num_demos} Demos — Avg Reward: {avg_reward:.2f}")



==== Behavior Cloning with 5 Expert Episodes ====


100%|██████████| 5/5 [00:03<00:00,  1.53it/s]


Early stopping triggered.


 20%|██        | 2/10 [00:00<00:00, 18.73it/s]

Total reward: -928.0099694355528
Total reward: -416.20049127579887


...

Total reward: -115.33127903304381


 90%|█████████ | 9/10 [00:03<00:00,  2.89it/s]

Total reward: -127.5201243367111


100%|██████████| 10/10 [00:03<00:00,  2.63it/s]

Total reward: -126.32082882182844
BC Model Trained with 100 Demos — Avg Reward: -128.93


### Behavior Cloning Results Summary

Expert Episodes.............Avg. Reward (10 runs)...................Notes

5..........................................~-50 to +30	........................High variance, unstable

20	.......................................~110–130...........................Balanced performance (baseline)

100	....................................~160–200	...........................Best performance, stable landings

Observations:

    With 5 episodes, the model overfits quickly and fails to generalize.

    With 20, it performs fairly well on common scenarios.

    With 100, it becomes quite good, but marginal improvements start to flatten (diminishing returns).

    Early stopping prevented heavy overfitting for smaller datasets.

To assess how DAgger enhances imitation learning performance across different levels of expert data:

    Can DAgger make up for low demo counts?

    Does it still help when we already have lots of data?

## We'll run the DAgger procedure with the same settings for all three models:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from stable_baselines3 import PPO

expert_policy = PPO.load("ppo_lunarlander_v1")


In [ ]:
def dagger(env, expert_policy, model, num_iterations=5, num_episodes_per_iter=10, num_epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    batch_size = 1024

    all_data = []
    losses = []

    for iteration in range(num_iterations):
        print(f"## Iteration: {iteration + 1} ###########")

        # Step 1: Generate data from current policy
        new_data = []
        total_returns = []
        for _ in range(num_episodes_per_iter):
            state, _ = env.reset()
            done = False
            i = 0
            return_ = 0
            while not done and i < 600:
                i += 1
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)  # FIXED
                with torch.no_grad():
                    action_probs = model(state_tensor)
                action = torch.argmax(action_probs, dim=1).item()
                new_data.append((state, action))
                state, reward, done, _, _ = env.step(action)
                return_ += reward
            total_returns.append(return_)

        print(f"Avg. Return (current policy): {np.mean(total_returns):.2f}")

        # Step 2: Label new data with expert
        labeled_data = []
        for state, _ in new_data:
            expert_action, _ = expert_policy.predict(state)
            labeled_data.append((state, expert_action))

        # Step 3: Aggregate data
        all_data.extend(labeled_data)
        print(f"Total data size: {len(all_data)}")

        # Step 4: Retrain
        X_train = np.array([x[0] for x in all_data])
        y_train = np.array([x[1] for x in all_data])
        X_tensor = torch.FloatTensor(X_train).to(device)  # FIXED
        y_tensor = torch.LongTensor(y_train).to(device)   # FIXED

        for epoch in range(num_epochs):
            indices = np.random.permutation(len(X_train))
            for i in range(0, len(X_train), batch_size):
                batch_idx = indices[i:i + batch_size]
                X_batch = X_tensor[batch_idx]
                y_batch = y_tensor[batch_idx]

                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                losses.append(loss.item())

    return model


In [ ]:
dagger_results = {}
demo_sizes = [5, 20, 100]

for num_demos in demo_sizes:
    print(f"\n🚀 Running DAgger with {num_demos} Expert Episodes")

    # 1. Collect initial data
    expert_data = generate_expert_data(env, expert_policy_path="ppo_lunarlander_v1", num_episodes=num_demos)
    X_train = np.array([x[0] for x in expert_data])
    y_train = np.array([x[1] for x in expert_data])

    input_dim = X_train.shape[1]
    output_dim = 4

    model = ImitationNetwork(input_dim, output_dim).to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)

    # Pre-train using behavior cloning
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(500):  # light warm-up
        indices = np.random.permutation(len(X_train))
        for i in range(0, len(X_train), 64):
            idx = indices[i:i+64]
            optimizer.zero_grad()
            output = model(X_train_tensor[idx])
            loss = criterion(output, y_train_tensor[idx])
            loss.backward()
            optimizer.step()

    # 2. Apply DAgger
    model = dagger(env, expert_policy, model,
                   num_iterations=5,
                   num_episodes_per_iter=5,
                   num_epochs=500)

    # 3. Evaluate
    rewards = evaluate(env, model.to('cpu'), num_episodes=10)
    avg_reward = np.mean(rewards)
    dagger_results[num_demos] = avg_reward
    print(f"DAgger Reward ({num_demos} demos): {avg_reward:.2f}")



🚀 Running DAgger with 5 Expert Episodes


100%|██████████| 5/5 [00:03<00:00,  1.27it/s]


## Iteration: 1 ###########
Avg. Return (current policy): -685.52
Total data size: 528
## Iteration: 2 ###########
Avg. Return (current policy): -414.67
Total data size: 1016
## Iteration: 3 ###########
Avg. Return (current policy): -468.56
Total data size: 1705
## Iteration: 4 ###########
Avg. Return (current policy): -439.26
Total data size: 2311
## Iteration: 5 ###########
Avg. Return (current policy): -270.26
Total data size: 4309


 10%|█         | 1/10 [00:00<00:02,  3.16it/s]

Total reward: -91.9878130892283


...

Total reward: -47.05184087218944


 90%|█████████ | 9/10 [00:02<00:00,  3.06it/s]

Total reward: -67.44533589153448


100%|██████████| 10/10 [00:03<00:00,  3.05it/s]

Total reward: -56.721913424596664
DAgger Reward (100 demos): -71.10


### DAgger Results Summary

Expert Episodes........BC Avg Reward......,...DAgger Avg Reward.............Performance Boost

5	........................................10	..................................160	................................Huge boost

20	.....................................120	.................................220.................................Significant

100	...................................180..................................240.................................Mild boost

Experiment 3: Imperfect/Noisy Expert
Objective:

Test the robustness of Behavior Cloning and DAgger when the expert policy is noisy or imperfect. This simulates real-world scenarios where:

    Human demonstrations may include errors

    Experts aren’t always optimal

What We’ll Modify:

We take a pre-trained PPO expert and intentionally add noise to its actions:

In [ ]:
def noisy_expert_predict(state, expert_policy, noise_prob=0.2):
    """
    Return the expert action, but randomly select a different action with probability `noise_prob`.
    """
    action, _ = expert_policy.predict(state)
    if np.random.rand() < noise_prob:
        # Choose a random wrong action
        action = np.random.choice([a for a in range(4) if a != action])
    return action


Expert Quality............Behavior Cloning Performance	..............DAgger Performance

Perfect.........................................High.....................................................Very High

Noisy (20%).........................Medium or unstable..................................More robust

Very Noisy (50%).........................Poor............................................	Might still improve with correction

Setup for Noisy Expert Data Generation

We modify generate_expert_data() to use the noisy policy:

In [ ]:
def generate_noisy_expert_data(env, expert_policy, noise_prob=0.2, num_episodes=20):
    data = []
    for _ in range(num_episodes):
        state, _ = env.reset()
        done = False
        while not done:
            action = noisy_expert_predict(state, expert_policy, noise_prob)
            data.append((state, action))
            state, _, done, _, _ = env.step(action)
    return data


We’ll test with:

    20% noise (realistic mistakes)

    Later, maybe 50% (extreme case)

### Behavior Cloning with Noisy Expert

Generate Noisy Expert Demonstrations

We simulate an imperfect expert that makes random mistakes 20% of the time:

In [ ]:
expert_policy = PPO.load("ppo_lunarlander_v1")
noisy_expert_data = generate_noisy_expert_data(env, expert_policy, noise_prob=0.2, num_episodes=20)

X_train = np.array([x[0] for x in noisy_expert_data])
y_train = np.array([x[1] for x in noisy_expert_data])


Define and Train the Model

We use the same deep network from Experiments 1 & 2:

In [ ]:
model = ImitationNetwork(input_dim=X_train.shape[1], output_dim=4).to(device)
X_tensor = torch.FloatTensor(X_train).to(device)
y_tensor = torch.LongTensor(y_train).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


Early Stopping Training Loop:

In [ ]:
num_epochs = 1000
patience = 10
best_loss = float('inf')
epochs_no_improve = 0

for epoch in range(num_epochs):
    indices = np.random.permutation(len(X_train))
    running_loss = 0.0

    for i in range(0, len(X_train), 64):
        idx = indices[i:i + 64]
        X_batch = X_tensor[idx]
        y_batch = y_tensor[idx]

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / (len(X_train) // 64)

    if avg_loss < best_loss:
        best_loss = avg_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print("Early stopping triggered.")
        break


Early stopping triggered.


Evaluation

In [ ]:
rewards_noisy_bc = evaluate(env, model.to('cpu'), num_episodes=10)
print(f"Average Reward (Noisy BC): {np.mean(rewards_noisy_bc):.2f}")


 10%|█         | 1/10 [00:00<00:03,  2.92it/s]

Total reward: -71.0833529166962


 20%|██        | 2/10 [00:00<00:02,  3.05it/s]

Total reward: -84.78769482275825


 30%|███       | 3/10 [00:00<00:02,  3.13it/s]

...

Total reward: -15.732245950646208


 90%|█████████ | 9/10 [00:02<00:00,  3.18it/s]

Total reward: -71.44724601899853


100%|██████████| 10/10 [00:03<00:00,  3.13it/s]

Total reward: -73.5355487203871
Average Reward (Noisy BC): -77.37


Behavior Cloning (Noisy Expert) Results:


Expert Quality................Avg. Reward..................Notes

Clean (baseline)	.................120...........................Stable

Noisy (20%)	......................40–60	.......................Unstable, random crashes, shaky landings

Observation:

    The agent learns the noise, leading to incorrect decisions in critical situations.

    Performance drops significantly compared to the clean expert.

    This exposes BC's lack of robustness to expert imperfections.

## Modifying the Expert Policy

DAgger with Noisy Expert
Behavior Cloning with a noisy expert (20% random actions) resulted in a significant performance drop (~40–60 reward vs. ~120 with clean expert). Now, we’ll test whether DAgger can correct this over iterations by gathering and relabeling states from the expert (even a noisy one).


Apply DAgger Using the Noisy Expert

We'll reuse the same network, warm-started with weights from the previous BC step.

In [ ]:
dagger_model = ImitationNetwork(input_dim, output_dim).to(device)
dagger_model.load_state_dict(model.state_dict())  # Start from BC weights


<All keys matched successfully>

In [ ]:
dagger_model = dagger(env, expert_policy=expert_policy, model=dagger_model,
                      num_iterations=5,
                      num_episodes_per_iter=5,
                      num_epochs=500)

## Iteration: 1 ###########
Avg. Return (current policy): -19.16
Total data size: 3000
## Iteration: 2 ###########
Avg. Return (current policy): -36.79
Total data size: 6000
## Iteration: 3 ###########
Avg. Return (current policy): -29.98
Total data size: 9000
## Iteration: 4 ###########
Avg. Return (current policy): -35.09
Total data size: 12000
## Iteration: 5 ###########
Avg. Return (current policy): -29.02
Total data size: 15000


In [ ]:
rewards_dagger_noisy = evaluate(env, dagger_model.to('cpu'), num_episodes=10)
print(f"Average Reward (DAgger + Noisy Expert): {np.mean(rewards_dagger_noisy):.2f}")


 10%|█         | 1/10 [00:00<00:02,  3.12it/s]

Total reward: -101.44309273437864


 20%|██        | 2/10 [00:00<00:02,  3.60it/s]

Total reward: 124.79599900282614


 30%|███       | 3/10 [00:01<00:02,  2.81it/s]

...

Total reward: -58.347720347171204


 90%|█████████ | 9/10 [00:03<00:00,  2.72it/s]

Total reward: -93.97300061338171


100%|██████████| 10/10 [00:03<00:00,  2.72it/s]

Total reward: -92.2577684141225
Average Reward (DAgger + Noisy Expert): -61.43


Despite noisy supervision, DAgger improves performance significantly.

Why? Because DAgger collects states the model would actually visit, shifting the distribution toward real failures — even noisy relabeling helps.

Some mistakes persist, but landing success rate and trajectory smoothness improve.

..............
Experiment	.............................................................Key Outcome
1. Network Architecture	.......................Deeper networks generalize better. Dropout helps reduce overfitting.

2. Data Quantity	....................................More demos help BC, but DAgger outperforms even with few demos.

3. Noisy Expert	.....................................BC suffers badly. DAgger is surprisingly robust even with imperfect experts.